In [123]:
import sys
sys.path.append("../utils/")
from runner import Runner
import pandas as pd

In [121]:
class FilterVariants:
    """Calculates duration of each case from the event table
    and appends to case table in column 'case:duration'
    
    This class provides functions that returns filtered case and event tables;
    filtering based on min flops, min duration, worst flops or both min flops and duration
    
    """
    
    def __init__(self,case_table,event_table):
        """
        INPUT: Case table and event table.
        Init appends 'case:duration' to the case table"""
        
        self.event_table = event_table
        case_durations = self.get_trace_durations()
        self.case_table = case_durations.merge(case_table, on="case:concept:name")

             
    def get_trace_durations(self):
        """calculates duration of each case from event table"""
        dfs= self.event_table.drop_duplicates('case:concept:name', keep='first')[['case:concept:name','timestamp:start']]
        dfe= self.event_table.drop_duplicates('case:concept:name', keep='last')[['case:concept:name','timestamp:end']]
        dfm = dfs.merge(dfe, on='case:concept:name')
        dfm['case:duration'] = dfm.apply(lambda row: row['timestamp:end']-row['timestamp:start'], axis=1)
        dfm = dfm.rename(columns={
            'timestamp:start':'case:timestamp:start',
            'timestamp:end':'case:timestamp:end',
        })
        return dfm
    
    def filter_best_flops(self):
        """returns case and event table corresponding to min flop instances"""
        best_flops_case = self.case_table[self.case_table['case:flops']==self.case_table['case:flops'].min()]
        filter_algs = best_flops_case['case:concept:name'].values
        best_flops_events = self.event_table.loc[self.event_table['case:concept:name'].isin(filter_algs)]
        return (best_flops_case, best_flops_events)
    
    def filter_worst_flops(self):
        """returns case and event table corresponding to max flop instances"""
        worst_flops_case = self.case_table[self.case_table['case:flops']==self.case_table['case:flops'].max()]
        filter_algs = worst_flops_case['case:concept:name'].values
        worst_flops_events = self.event_table.loc[self.event_table['case:concept:name'].isin(filter_algs)]
        return (worst_flops_case, worst_flops_events)
    
    def filter_best_duration(self):
        """returns case and event table corresponding to min duration instances"""
        best_duration_case = self.case_table[self.case_table['case:duration']==self.case_table['case:duration'].min()]
        filter_algs = best_duration_case['case:concept:name'].values
        best_duration_events = self.event_table.loc[self.event_table['case:concept:name'].isin(filter_algs)]
        return (best_duration_case, best_duration_events)
    
    
    def filter_best_flops_duration(self):
        """returns case and event table corresponding to min flop and min duration instances
        This function appends 'case:rel-flops' (relative flops: flop-min_flop) and 
        'case:rel-duration' (relative duration: duration - min_duration) to the returned case table.
        
        if case:rel-flops = case:rel-duration = 0, then minimum flops implies minimum duration.
        
        """
        
        best_flops_case = self.case_table[self.case_table['case:flops']==self.case_table['case:flops'].min()]
        best_duration_case = self.case_table[self.case_table['case:duration']==self.case_table['case:duration'].min()]
        df = pd.concat([best_flops_case, best_duration_case]).drop_duplicates('case:concept:name')
        
        min_flop = best_flops_case['case:flops'].values[0]
        df['case:rel-flops'] = df.apply(lambda row: row['case:flops']-min_flop, axis=1)
        
        min_duration = best_duration_case['case:duration'].values[0]
        df['case:rel-duration'] = df.apply(lambda row: row['case:duration']-min_duration, axis=1)
        
        filter_algs = df['case:concept:name'].values
        df_events = self.event_table.loc[self.event_table['case:concept:name'].isin(filter_algs)]
        
        return (df, df_events)
    
    def clear_tables(self):
        self.case_table = None
        self.event_table = None
        
        

In [114]:
runner = Runner("Matrix-Chain", "../Matrix-Chain-4/variants-linnea/generate-variants-linnea.py")
args = ["10","110","120","130","500"]
ret = runner.generate_run_experiments(args,bRun=True, bGenerate=False)
ct,et,rt = runner.get_all_tables(runner.get_exp_dir(args), meta=True)

Running Experiments
Experiments completed


In [115]:
filterVariants = FilterVariants(ct,rt)

In [119]:
filterVariants.case_table

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
0,algorithm1,1.654778e+09,1.654778e+09,0.005248,14900000.0,3
1,algorithm5,1.654778e+09,1.654778e+09,0.001278,29100000.0,3
2,algorithm4,1.654778e+09,1.654778e+09,0.000553,29100000.0,3
3,algorithm0,1.654778e+09,1.654778e+09,0.000425,2710000.0,3
4,algorithm3,1.654778e+09,1.654778e+09,0.000625,18000000.0,3
5,algorithm2,1.654778e+09,1.654778e+09,0.000993,17900000.0,3


In [116]:
ct_best_ft, et_best_ft = filterVariants.filter_best_flops_duration()


In [117]:
ct_best_ft

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels,case:rel-flops,case:rel-duration
3,algorithm0,1.654778e+09,1.654778e+09,0.000425,2710000.0,3,0.0,0.0


In [118]:
et_best_ft

,case:concept:name,concept:name,concept:kernel,timestamp:start,timestamp:end
9,algorithm0,gemm_3.12e+05,"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",1.654778e+09,1.654778e+09
10,algorithm0,gemm_1.3e+06,"gemm!('N', 'N', 1.0, ml4, ml3, 0.0, ml5)",1.654778e+09,1.654778e+09
11,algorithm0,gemm_1.1e+06,"gemm!('N', 'N', 1.0, ml0, ml5, 0.0, ml6)",1.654778e+09,1.654778e+09


In [105]:
ct_best_dur, et_best_dur = filterVariants.filter_best_duration()

In [106]:
ct_best_dur

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
3,algorithm0,1.654777e+09,1.654777e+09,0.000414,2710000.0,3


In [107]:
et_best_dur

,case:concept:name,concept:name,concept:kernel,timestamp:start,timestamp:end
9,algorithm0,gemm_3.12e+05,"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",1.654777e+09,1.654777e+09
10,algorithm0,gemm_1.3e+06,"gemm!('N', 'N', 1.0, ml4, ml3, 0.0, ml5)",1.654777e+09,1.654777e+09
11,algorithm0,gemm_1.1e+06,"gemm!('N', 'N', 1.0, ml0, ml5, 0.0, ml6)",1.654777e+09,1.654777e+09


In [74]:
ct_worst_flops, et_worst_flops = filterVariants.filter_worst_flops()

In [82]:
ct_worst_flops

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
1,algorithm5,1.654777e+09,1.654777e+09,0.001046,29100000.0,3
2,algorithm4,1.654777e+09,1.654777e+09,0.000537,29100000.0,3


In [83]:
et_worst_flops

,case:concept:name,concept:name,concept:kernel,timestamp:start,timestamp:end
3,algorithm5,gemm_2.64e+05,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml4)",1.654777e+09,1.654777e+09
4,algorithm5,gemm_1.56e+07,"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml5)",1.654777e+09,1.654777e+09
5,algorithm5,gemm_1.32e+07,"gemm!('N', 'N', 1.0, ml4, ml5, 0.0, ml6)",1.654777e+09,1.654777e+09
6,algorithm4,gemm_1.56e+07,"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml4)",1.654777e+09,1.654777e+09
7,algorithm4,gemm_2.64e+05,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml5)",1.654777e+09,1.654777e+09
8,algorithm4,gemm_1.32e+07,"gemm!('N', 'N', 1.0, ml5, ml4, 0.0, ml6)",1.654777e+09,1.654777e+09


In [85]:
pd.concat([ct_best_dur,ct_worst_flops])

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
3,algorithm0,1.654777e+09,1.654777e+09,0.000414,2710000.0,3
1,algorithm5,1.654777e+09,1.654777e+09,0.001046,29100000.0,3
2,algorithm4,1.654777e+09,1.654777e+09,0.000537,29100000.0,3


In [86]:
ct_best_flops, et_best_flops = filterVariants.filter_best_flops()

In [124]:
ct

,case:concept:name,case:flops,case:num_kernels
0,algorithm1,14900000.0,3
1,algorithm5,29100000.0,3
2,algorithm4,29100000.0,3
3,algorithm0,2710000.0,3
4,algorithm3,18000000.0,3
5,algorithm2,17900000.0,3


In [87]:
ct_best_flops


,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
3,algorithm0,1.654777e+09,1.654777e+09,0.000414,2710000.0,3
